In [2]:
import bs4
import requests
from urllib.request import Request, urlopen
import re
import pandas as pd
import time
import os
import json
import datetime 
import concurrent.futures as futures
import difflib
from random import randint
from time import sleep
from seleniumrequests import Chrome



try:
    from googlesearch import search
except ImportError: 
    print("No module named 'google' found")
from IPython.core.display import display, HTML

In [3]:
all_prob_tags = ['a','span','p','h1','strong']

In [4]:
os.chdir(r'E:\Competitions\generic scrapper')

In [5]:
api_key = open(r'E:\Personal documents\Passwords\googleapikey.txt').read()

item_list = ['shoe','wallet','mobile','watch','toy','makeup','Mobile','Telivision','Laptop','Camera','Apparel','Jwellery','Book','Music','VideoGame','Kitchen','Bike','Car','Home','Decor']

In [6]:
# resp = requests.get(f'https://www.google.com/search?q={item_list[0]}&tbm=shop&key={api_key}')

In [7]:
# all_products = []
# for item in item_list:
#     resp = requests.get(f'https://www.google.com/search?q={item}&tbm=shop&key={api_key}')
#     bs_object = bs4.BeautifulSoup(resp.content,'lxml')
#     for prod in bs_object.find_all('a',href = True):
        
#         if prod['href'].startswith('http'):
#             all_products.append(prod['href'])
#         else:
#             pass
# prod_page = page_reader(all_products[140])
# prod_bs_object = bs4.BeautifulSoup(prod_page,'lxml')
# # prod_bs_object.find_all('span',id = re.compile('(Title|title|TITLE)'),text = re.compile('Wallet'))
# prod_bs_object.find(text = re.compile('Wallet'))

In [8]:
# # bs_object.find_all('div',{'class':'sh-np__product-title translate-content'})
# prod_dict = {}
# for item in item_list:
#     resp = requests.get(f'https://www.google.com/search?q={item}&tbm=shop&key={api_key}')
#     bs_object = bs4.BeautifulSoup(resp.content,'lxml')
#     for ob in bs_object.find_all('div',{'class': 'KZmu8e'}):
#         website = ob.find('div', {'class':'sh-np__seller-container'}).text
#         title = ob.find('div', {'class':'sh-np__product-title translate-content'}).text
#         price = ob.find('span', {'class':'T14wmb'}).text
#         url = ob.find('a',href = True)['href']
        
#         prod_dict[website] = [url,title,price,url]

In [9]:
# ##Save dict as json
# crnt_time= re.sub('\D+','',str(datetime.datetime.now())[:16])
# print(crnt_time)
# out_file = open(f"E:/Competitions/generic scrapper/website_output_{crnt_time}.json", "w")
# json.dump(prod_dict,out_file)

In [10]:
f = open('website_output_202106182319.json')
prod_dict = json.load(f)

In [11]:
del prod_dict['Myntra']

In [12]:


def timeout(timelimit):
    def decorator(func):
        def decorated(*args, **kwargs):
            with futures.ThreadPoolExecutor(max_workers=1) as executor:
                future = executor.submit(func, *args, **kwargs)
                try:
                    result = future.result(timelimit)
                except futures.TimeoutError:
                    print('Timedout!')
                    raise TimeoutError from None
                else:
                    pass
                executor._threads.clear()
                futures.thread._threads_queues.clear()
                return result
        return decorated
    return decorator



In [13]:
@timeout(10)
def page_reader(url):
    try:
        header = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'}

        r = requests.get(f"http://webcache.googleusercontent.com/search?q=cache:{url}",headers=header)
        if len(r.content)<5000:
            raise ValueError('Trying with urlopen')
        else:
            return(r.content)
    except Exception as ex:
        print(ex)
        req = Request(url,headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'})
        webpage = urlopen(req).read()
        return(webpage)

In [17]:
# ##Save dict as json
# crnt_time= re.sub('\D+','',str(datetime.datetime.now())[:16])
# print(crnt_time)
# web_out_file = open(f"E:/Competitions/generic scrapper/website_content_{crnt_time}.json", "w")
# json.dump(page_reader_dict,web_out_file)

In [18]:
def prod_title(bs_object,prod_class,tag_id):
    title = bs_object.find(prod_class,{'id' : tag_id}).text.replace('\n','').strip()
    return(title)

In [19]:
def prod_code(bs_object,prod_class,tag_id):
    code = bs_object.find(prod_class,{'id' : tag_id})

In [20]:
def substring_matcher(search_string,bs_object):
    if len(search_string)<=5:
        return(None)
    
    search_result_tag = bs_object.find(string = re.compile(f'{search_string}.*?'))
    if search_result_tag:
        return(search_result_tag)
    else:
        substring_matcher(search_string[:int(len(search_string)/2)],bs_object)
        

In [21]:
def check_float(potential_float):
    try:
        float(potential_float)
        return True
    except ValueError:
        return False

In [1]:
title_tags_attrs = {}
for i,site in enumerate(prod_dict.keys()):
    try:
        web_page = page_reader(prod_dict[site][0])
        prod_bs_object = bs4.BeautifulSoup(web_page,'lxml')
    #         print(substring_matcher(prod_dict[site][1],prod_bs_object))
        prod_all_text = [' '.join(tag.text.split()) for  tag in prod_bs_object.find_all(all_prob_tags)]
        text = prod_dict[site][1]
        best_match = difflib.get_close_matches(text,prod_all_text,cutoff = 0.1)[0]
        
        tag_name = prod_bs_object.find_all(all_prob_tags,string = best_match)[0].name
        tag_attrs = prod_bs_object.find_all(all_prob_tags,string = best_match)[0].attrs
        
    #         print(prod_bs_object.find(text = prod_dict[site][1]))
    #         print(prod_bs_object.find(text = prod_dict[site][2]))
        
        #####Creating tags list for price
        price_text = prod_dict[site][2]
        prod_price_text = [k for k in prod_all_text if 'Rs' in k]
        
        print(prod_price_text)
        
        title_tags_attrs[site] = [tag_name,tag_attrs]

    except Exception as ex:
        print(i)
        print(site)
        print(ex)
        pass

In [22]:
currency_file = open('currencies.json','r')
currencies_dict = json.load(currency_file)
currency_file.close()
all_curr_symbols = [curr['symbol'] for curr in currencies_dict] + [curr['cc'] for curr in currencies_dict]


In [23]:
# title_tags_attrs['common'] = ['title']

In [24]:
# crnt_time= re.sub('\D+','',str(datetime.datetime.now())[:16])
# title_json  = open(f'title_mapper_{crnt_time}.json','w')
# json.dump(title_tags_attrs,title_json)

In [25]:
crnt_time= re.sub('\D+','',str(datetime.datetime.now())[:16])
title_file =open('title_mapper_202106191704.json','r')
title_tags_attrs = json.load(title_file)

In [26]:
def title_finder(prod_bs_object):
    all_tags_out = []
    for conf in title_tags_attrs.keys():
            try:
                tag_out = prod_bs_object.find(title_tags_attrs[conf][0]).text

                
                all_tags_out.append(tag_out)
            except Exception as ex:
                pass
    prod_title = max(set(all_tags_out), key=all_tags_out.count).replace(u'\xa0',u' ').strip()
    return(prod_title)

# Finding the price

In [27]:
def price_finder(prod_bs_object,all_curr_symbols):
    prod_all_text = [' '.join(tag.text.split()) for  tag in prod_bs_object.find_all(all_prob_tags)]
    currency_counter = dict((k,0) for k in all_curr_symbols)
    for sym in all_curr_symbols:

        if f'{sym}' in ' '.join(prod_all_text):

            currency_counter[sym]+=1
    all_prob_price = []
    while len(all_prob_price)==0 and max(currency_counter.values())>0:
        currency = max(currency_counter, key=currency_counter.get)
        all_price_text = [' '.join(x.text.split()) for x in prod_bs_object.find_all(all_prob_tags + ['div']) if currency in x.text]

        for prob_price in all_price_text:
            prob_price = prob_price.replace(currency,'')
            prob_price = re.sub('[^a-zA-Z0-9.]','',prob_price)

            prob_price = re.sub('^\.','',prob_price)

            if check_float(prob_price):
                if float(prob_price)>0:
                    all_prob_price.append(prob_price)

        currency_counter.pop(currency)
    price = currency + max(set(all_prob_price), key=all_prob_price.count).replace(u'\xa0',u' ')
    return(price)

# Finding SKU

In [74]:
def sku_finder(page_content):
  
    sku_list = []
    black_list = ['content','quot','\n']
    for i in re.findall('sku.{5,20}?[^a-zA-Z0-9.-]',re.sub('|'.join(black_list),'',str(page_content))):
        
        prob_sku = re.sub('[^a-zA-Z0-9.-]','',i).replace("sku",'')
        if prob_sku!='':
            sku_list.append(prob_sku)
    if not sku_list:
        for i in re.findall('(sku).{5,20}?[^a-zA-Z0-9.-]',str(page_content)):
            if prob_sku!='':
                sku_list.append(re.sub("product.id",'',re.sub('[^a-zA-Z0-9.-]','',re.sub('|'.join(black_list),'',i))))

    if len(sku_list)==len(set(sku_list)):

        return(max(sku_list,key = len))
    else:
        return(max(set(sku_list), key=sku_list.count))

In [75]:
all_test_urls = pd.read_csv('ProdDetails_Link.txt',header = None)


In [76]:
output_dict = {}
for test_url in all_test_urls[0]:
# for test_url in page_reader_dict.keys():
    print(test_url)
#     this_seconds = randint(5,30)
#     print(f'Sleeping for {this_seconds} seconds to avoid bot detection')
#     sleep(this_seconds)
    
    try:
        page_content = page_reader(test_url)
#         page_content = page_reader_dict[test_url]
        prod_bs_object = bs4.BeautifulSoup(page_content,'lxml')
        try:
            title = (title_finder(prod_bs_object))
        except:
            title = ''
        
        try:    
            price = price_finder(prod_bs_object,all_curr_symbols)
        except:
            price = ''
        
        try:
            sku = sku_finder(page_content)
        except:
            sku = ''
        print(title,price,sku)
        output_dict[test_url] = [title,price,sku]
    except Exception as ex:
        print(ex)
        

https://www.missguidedau.com/tall-blue-dark-wash-button-through-fitted-denim-dress-10153103
Trying with urlopen
Tall Blue Dark Wash Button Through Fitted Denim Dress $100.99 10153103006
https://www.next.com.au/en/g968006s1/122806#122806
Trying with urlopen
Cotton Curtains $64 122-806
https://www.catch.com.au/product/sheraton-luxury-waterproof-queen-bed-mattress-protector-6427445/
Trying with urlopen
Sheraton Luxury Waterproof Queen Bed Mattress Protector $29.99 -id
https://www.zara.com/au/en/chunky-black-shoes-p12424620.html?v1=86429636&v2=1736750
Trying with urlopen
CHUNKY BLACK SHOES AUD119.00 86429636-040-44
https://www.thegoodguys.com.au/fridges-and-freezers/refrigerators
Trying with urlopen
Refrigerators $995.00 50071344
https://www.cos.net.au/technology/projectors-and-accessories/Epson-Projector-EB2250U-PROJ9539
Trying with urlopen
Epson Projector EB-2250U  n
https://www.chemistwarehouse.com.au/shop-online/258/medicines
Trying with urlopen
Medicines  Id.val
https://au.tesalate.co

In [ ]:
# page_content = page_reader_dict['https://www.chemistwarehouse.com.au/shop-online/258/medicines']
# prod_bs_object = bs4.BeautifulSoup(page_content,'lxml')

In [78]:
crnt_time= re.sub('\D+','',str(datetime.datetime.now())[:16])
output_df = pd.DataFrame.from_dict(output_dict, orient='index').reset_index(drop = False)
output_df.columns = ['Url','Title','Price','SKU']
output_df.to_csv(f'TestLinkOutput_{crnt_time}.csv')

In [63]:
test_url = 'https://www.myer.com.au/p/lego-creator-ford-mustang'
page_content = page_reader(test_url)
prod_bs_object = bs4.BeautifulSoup(page_content,'lxml')

title = (title_finder(prod_bs_object))
print(title)
sku = sku_finder(page_content)
print(sku)
price = price_finder(prod_bs_object,all_curr_symbols)

Trying with urlopen
LEGO Creator Ford Mustang 10265


ValueError: max() arg is an empty sequence